# Optimizing Performance using PARTITIONED BY and CLUSTERED BY
<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->

This notebook is a review of Apache Druid's data sharding strategy and how it can be used to improve system health and query performance.

Apache Druid stores data in Segment files. Druid Segments have a columnar structure designed for high performance analytic queries. Segments are organized into consistent time intervals called the `Segment Granularity` of the table and typically expressed as HOUR, DAY, MONTH, or YEAR. In this notebook we will review how this setting affects the number of segments created during ingestion and the size of those segments. Druid works best when segments are balanced in size and cover approximately 5 million rows each. 

Pruning is the process of reducing the segment files that need to be inspected in order to resolve a query. To process a query in Druid, all the time chunks that overlap the query's filter condition on the __time column will be inspected. 

Within is time chunk, segment files can be organized based on another set of columns. In SQL Based ingestion it's specified using a CLUSTERED BY clause. When you use this feature, Druid will organize the segments within a time chuck such that each one covers a range of the values. At query time the Broker can do additional pruning of the segments within a time chunk if the user specifies a filter condition on the clustering columns. 

Streaming ingestion is inherently different because it is optimized for scalable throughput. Streaming ingestion scales up by adding more tasks. Each task ingests portions of the data from a stream and generates segment files idependently from other tasks. With more parallel tasks, this causes more fragmentation. We will review how compaction tasks are used to merge segments and how compaction can apply a secondary partitioning strategy in order to optimize the data after ingestion.


## Prerequisites

This tutorial works was tested with Druid 27.0.0.

#### Run with Docker

<!-- Profiles are:
`druid-jupyter` - just Jupyter and Druid
`all-services` - includes Jupyter, Druid, and Kafka
 -->

Launch this tutorial and all prerequisites using the `all-services` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see [the project on github](https://github.com/implydata/learn-druid).
   

## Initialization

The following cells set up the notebook and learning environment ready for use.

### Set up and connect to the learning environment

Run the next cell to set up the Druid Python client's connection to Apache Druid.

If successful, the Druid version number will be shown in the output.

In [1]:
import druidapi
import os

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"
    
print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)

display = druid.display
sql_client = druid.sql
status_client = druid.status

status_client.version

Opening a connection to http://router:8888.


'27.0.0-SNAPSHOT'

<!-- Include these cells if your notebook uses Kafka. -->

Run the next cell to set up the connection to Apache Kafka.

In [2]:
if 'KAFKA_HOST' not in os.environ.keys():
   kafka_host=f"http://localhost:9092"
else:
    kafka_host=f"{os.environ['KAFKA_HOST']}:9092"

## Time Partitioning

Normally, ideal segments contain around 5 million rows, but given that this is all running on a laptop and for fast demonstration purposes, we'll set our "ideal" to only 50000 rows.


### Too Granular
Using a `Segment Granularity` that is too small will render too many segments. The following batch ingestion demonstrates this. It will take about 1 minute to complete:

In [4]:
sql='''
REPLACE INTO "flights_hour" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://static.imply.io/example-data/flight_on_time/flights/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11.csv.zip"]}',
    '{"type":"csv","findColumnsFromHeader":true}'
  )
) EXTEND ("depaturetime" VARCHAR, "arrivalime" VARCHAR, "Year" BIGINT, "Quarter" BIGINT, "Month" BIGINT, "DayofMonth" BIGINT, "DayOfWeek" BIGINT, "FlightDate" VARCHAR, "Reporting_Airline" VARCHAR, "DOT_ID_Reporting_Airline" BIGINT, "IATA_CODE_Reporting_Airline" VARCHAR, "Tail_Number" VARCHAR, "Flight_Number_Reporting_Airline" BIGINT, "OriginAirportID" BIGINT, "OriginAirportSeqID" BIGINT, "OriginCityMarketID" BIGINT, "Origin" VARCHAR, "OriginCityName" VARCHAR, "OriginState" VARCHAR, "OriginStateFips" BIGINT, "OriginStateName" VARCHAR, "OriginWac" BIGINT, "DestAirportID" BIGINT, "DestAirportSeqID" BIGINT, "DestCityMarketID" BIGINT, "Dest" VARCHAR, "DestCityName" VARCHAR, "DestState" VARCHAR, "DestStateFips" BIGINT, "DestStateName" VARCHAR, "DestWac" BIGINT, "CRSDepTime" BIGINT, "DepTime" BIGINT, "DepDelay" BIGINT, "DepDelayMinutes" BIGINT, "DepDel15" BIGINT, "DepartureDelayGroups" BIGINT, "DepTimeBlk" VARCHAR, "TaxiOut" BIGINT, "WheelsOff" BIGINT, "WheelsOn" BIGINT, "TaxiIn" BIGINT, "CRSArrTime" BIGINT, "ArrTime" BIGINT, "ArrDelay" BIGINT, "ArrDelayMinutes" BIGINT, "ArrDel15" BIGINT, "ArrivalDelayGroups" BIGINT, "ArrTimeBlk" VARCHAR, "Cancelled" BIGINT, "CancellationCode" VARCHAR, "Diverted" BIGINT, "CRSElapsedTime" BIGINT, "ActualElapsedTime" BIGINT, "AirTime" BIGINT, "Flights" BIGINT, "Distance" BIGINT, "DistanceGroup" BIGINT, "CarrierDelay" BIGINT, "WeatherDelay" BIGINT, "NASDelay" BIGINT, "SecurityDelay" BIGINT, "LateAircraftDelay" BIGINT, "FirstDepTime" VARCHAR, "TotalAddGTime" VARCHAR, "LongestAddGTime" VARCHAR, "DivAirportLandings" VARCHAR, "DivReachedDest" VARCHAR, "DivActualElapsedTime" VARCHAR, "DivArrDelay" VARCHAR, "DivDistance" VARCHAR, "Div1Airport" VARCHAR, "Div1AirportID" VARCHAR, "Div1AirportSeqID" VARCHAR, "Div1WheelsOn" VARCHAR, "Div1TotalGTime" VARCHAR, "Div1LongestGTime" VARCHAR, "Div1WheelsOff" VARCHAR, "Div1TailNum" VARCHAR, "Div2Airport" VARCHAR, "Div2AirportID" VARCHAR, "Div2AirportSeqID" VARCHAR, "Div2WheelsOn" VARCHAR, "Div2TotalGTime" VARCHAR, "Div2LongestGTime" VARCHAR, "Div2WheelsOff" VARCHAR, "Div2TailNum" VARCHAR, "Div3Airport" VARCHAR, "Div3AirportID" VARCHAR, "Div3AirportSeqID" VARCHAR, "Div3WheelsOn" VARCHAR, "Div3TotalGTime" VARCHAR, "Div3LongestGTime" VARCHAR, "Div3WheelsOff" VARCHAR, "Div3TailNum" VARCHAR, "Div4Airport" VARCHAR, "Div4AirportID" VARCHAR, "Div4AirportSeqID" VARCHAR, "Div4WheelsOn" VARCHAR, "Div4TotalGTime" VARCHAR, "Div4LongestGTime" VARCHAR, "Div4WheelsOff" VARCHAR, "Div4TailNum" VARCHAR, "Div5Airport" VARCHAR, "Div5AirportID" VARCHAR, "Div5AirportSeqID" VARCHAR, "Div5WheelsOn" VARCHAR, "Div5TotalGTime" VARCHAR, "Div5LongestGTime" VARCHAR, "Div5WheelsOff" VARCHAR, "Div5TailNum" VARCHAR, "Unnamed: 109" VARCHAR))
SELECT
  TIME_PARSE("depaturetime") AS "__time",
  "arrivalime", "Year", "Quarter", "Month", "DayofMonth", "DayOfWeek", "FlightDate", "Reporting_Airline", "DOT_ID_Reporting_Airline",
  "IATA_CODE_Reporting_Airline", "Tail_Number", "Flight_Number_Reporting_Airline", 
  "OriginAirportID", "OriginAirportSeqID", "OriginCityMarketID", "Origin", "OriginCityName", "OriginState", "OriginStateFips", "OriginStateName", "OriginWac",  
  "DestAirportID", "DestAirportSeqID", "DestCityMarketID", "Dest", "DestCityName", "DestState", "DestStateFips", "DestStateName", "DestWac", 
  "CRSDepTime", "DepTime", "DepDelay", "DepDelayMinutes", "DepDel15", "DepartureDelayGroups", "DepTimeBlk",
  "TaxiOut", "WheelsOff", "WheelsOn", "TaxiIn", "CRSArrTime", "ArrTime", "ArrDelay", "ArrDelayMinutes", "ArrDel15", "ArrivalDelayGroups", "ArrTimeBlk",
  "Cancelled", "CancellationCode", "Diverted", "CRSElapsedTime", "ActualElapsedTime", "AirTime", "Flights", "Distance", "DistanceGroup", "CarrierDelay",
  "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay", "FirstDepTime", "TotalAddGTime", "LongestAddGTime", "DivAirportLandings", "DivReachedDest",
  "DivActualElapsedTime",  "DivArrDelay",  "DivDistance",  "Div1Airport",  "Div1AirportID",  "Div1AirportSeqID",  "Div1WheelsOn",  "Div1TotalGTime",
  "Div1LongestGTime",  "Div1WheelsOff",  "Div1TailNum",  "Div2Airport",  "Div2AirportID",  "Div2AirportSeqID",  "Div2WheelsOn",  "Div2TotalGTime",
  "Div2LongestGTime",  "Div2WheelsOff",  "Div2TailNum",  "Div3Airport",  "Div3AirportID",  "Div3AirportSeqID",  "Div3WheelsOn",  "Div3TotalGTime",
  "Div3LongestGTime",  "Div3WheelsOff",  "Div3TailNum",  "Div4Airport",  "Div4AirportID",  "Div4AirportSeqID",  "Div4WheelsOn",  "Div4TotalGTime",
  "Div4LongestGTime",  "Div4WheelsOff",  "Div4TailNum",  "Div5Airport",  "Div5AirportID",  "Div5AirportSeqID",  "Div5WheelsOn",  "Div5TotalGTime",
  "Div5LongestGTime",  "Div5WheelsOff",  "Div5TailNum"
FROM "ext"
PARTITIONED BY HOUR'''

display.run_task(sql)
sql_client.wait_until_ready('flights_hour')

Loading data, status:[SUCCESS]: 100%|██████████| 100.0/100.0 [01:05<00:00,  1.52it/s]            


Position,Name,Type
1,__time,TIMESTAMP
2,arrivalime,VARCHAR
3,Year,BIGINT
4,Quarter,BIGINT
5,Month,BIGINT
6,DayofMonth,BIGINT
7,DayOfWeek,BIGINT
8,FlightDate,VARCHAR
9,Reporting_Airline,VARCHAR
10,DOT_ID_Reporting_Airline,BIGINT


You can see the segments that were created on [the Apache Druid console in the Segments view](http://localhost:8888/unified-console.html#segments/datasource~flights_hour).

Here's what it should look like, notice that each segment corresponds to a single time interval of one hour and the number of rows in the segments are small and highly variable:
![](assets/segments-hourly.png)


### Too Coarse
In this example we overcorrect by using a granularity of YEAR:

In [6]:
sql='''
REPLACE INTO "flights_year" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://static.imply.io/example-data/flight_on_time/flights/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11.csv.zip"]}',
    '{"type":"csv","findColumnsFromHeader":true}'
  )
) EXTEND ("depaturetime" VARCHAR, "arrivalime" VARCHAR, "Year" BIGINT, "Quarter" BIGINT, "Month" BIGINT, "DayofMonth" BIGINT, "DayOfWeek" BIGINT, "FlightDate" VARCHAR, "Reporting_Airline" VARCHAR, "DOT_ID_Reporting_Airline" BIGINT, "IATA_CODE_Reporting_Airline" VARCHAR, "Tail_Number" VARCHAR, "Flight_Number_Reporting_Airline" BIGINT, "OriginAirportID" BIGINT, "OriginAirportSeqID" BIGINT, "OriginCityMarketID" BIGINT, "Origin" VARCHAR, "OriginCityName" VARCHAR, "OriginState" VARCHAR, "OriginStateFips" BIGINT, "OriginStateName" VARCHAR, "OriginWac" BIGINT, "DestAirportID" BIGINT, "DestAirportSeqID" BIGINT, "DestCityMarketID" BIGINT, "Dest" VARCHAR, "DestCityName" VARCHAR, "DestState" VARCHAR, "DestStateFips" BIGINT, "DestStateName" VARCHAR, "DestWac" BIGINT, "CRSDepTime" BIGINT, "DepTime" BIGINT, "DepDelay" BIGINT, "DepDelayMinutes" BIGINT, "DepDel15" BIGINT, "DepartureDelayGroups" BIGINT, "DepTimeBlk" VARCHAR, "TaxiOut" BIGINT, "WheelsOff" BIGINT, "WheelsOn" BIGINT, "TaxiIn" BIGINT, "CRSArrTime" BIGINT, "ArrTime" BIGINT, "ArrDelay" BIGINT, "ArrDelayMinutes" BIGINT, "ArrDel15" BIGINT, "ArrivalDelayGroups" BIGINT, "ArrTimeBlk" VARCHAR, "Cancelled" BIGINT, "CancellationCode" VARCHAR, "Diverted" BIGINT, "CRSElapsedTime" BIGINT, "ActualElapsedTime" BIGINT, "AirTime" BIGINT, "Flights" BIGINT, "Distance" BIGINT, "DistanceGroup" BIGINT, "CarrierDelay" BIGINT, "WeatherDelay" BIGINT, "NASDelay" BIGINT, "SecurityDelay" BIGINT, "LateAircraftDelay" BIGINT, "FirstDepTime" VARCHAR, "TotalAddGTime" VARCHAR, "LongestAddGTime" VARCHAR, "DivAirportLandings" VARCHAR, "DivReachedDest" VARCHAR, "DivActualElapsedTime" VARCHAR, "DivArrDelay" VARCHAR, "DivDistance" VARCHAR, "Div1Airport" VARCHAR, "Div1AirportID" VARCHAR, "Div1AirportSeqID" VARCHAR, "Div1WheelsOn" VARCHAR, "Div1TotalGTime" VARCHAR, "Div1LongestGTime" VARCHAR, "Div1WheelsOff" VARCHAR, "Div1TailNum" VARCHAR, "Div2Airport" VARCHAR, "Div2AirportID" VARCHAR, "Div2AirportSeqID" VARCHAR, "Div2WheelsOn" VARCHAR, "Div2TotalGTime" VARCHAR, "Div2LongestGTime" VARCHAR, "Div2WheelsOff" VARCHAR, "Div2TailNum" VARCHAR, "Div3Airport" VARCHAR, "Div3AirportID" VARCHAR, "Div3AirportSeqID" VARCHAR, "Div3WheelsOn" VARCHAR, "Div3TotalGTime" VARCHAR, "Div3LongestGTime" VARCHAR, "Div3WheelsOff" VARCHAR, "Div3TailNum" VARCHAR, "Div4Airport" VARCHAR, "Div4AirportID" VARCHAR, "Div4AirportSeqID" VARCHAR, "Div4WheelsOn" VARCHAR, "Div4TotalGTime" VARCHAR, "Div4LongestGTime" VARCHAR, "Div4WheelsOff" VARCHAR, "Div4TailNum" VARCHAR, "Div5Airport" VARCHAR, "Div5AirportID" VARCHAR, "Div5AirportSeqID" VARCHAR, "Div5WheelsOn" VARCHAR, "Div5TotalGTime" VARCHAR, "Div5LongestGTime" VARCHAR, "Div5WheelsOff" VARCHAR, "Div5TailNum" VARCHAR, "Unnamed: 109" VARCHAR))
SELECT
  TIME_PARSE("depaturetime") AS "__time",
  "arrivalime", "Year", "Quarter", "Month", "DayofMonth", "DayOfWeek", "FlightDate", "Reporting_Airline", "DOT_ID_Reporting_Airline",
  "IATA_CODE_Reporting_Airline", "Tail_Number", "Flight_Number_Reporting_Airline", 
  "OriginAirportID", "OriginAirportSeqID", "OriginCityMarketID", "Origin", "OriginCityName", "OriginState", "OriginStateFips", "OriginStateName", "OriginWac",  
  "DestAirportID", "DestAirportSeqID", "DestCityMarketID", "Dest", "DestCityName", "DestState", "DestStateFips", "DestStateName", "DestWac", 
  "CRSDepTime", "DepTime", "DepDelay", "DepDelayMinutes", "DepDel15", "DepartureDelayGroups", "DepTimeBlk",
  "TaxiOut", "WheelsOff", "WheelsOn", "TaxiIn", "CRSArrTime", "ArrTime", "ArrDelay", "ArrDelayMinutes", "ArrDel15", "ArrivalDelayGroups", "ArrTimeBlk",
  "Cancelled", "CancellationCode", "Diverted", "CRSElapsedTime", "ActualElapsedTime", "AirTime", "Flights", "Distance", "DistanceGroup", "CarrierDelay",
  "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay", "FirstDepTime", "TotalAddGTime", "LongestAddGTime", "DivAirportLandings", "DivReachedDest",
  "DivActualElapsedTime",  "DivArrDelay",  "DivDistance",  "Div1Airport",  "Div1AirportID",  "Div1AirportSeqID",  "Div1WheelsOn",  "Div1TotalGTime",
  "Div1LongestGTime",  "Div1WheelsOff",  "Div1TailNum",  "Div2Airport",  "Div2AirportID",  "Div2AirportSeqID",  "Div2WheelsOn",  "Div2TotalGTime",
  "Div2LongestGTime",  "Div2WheelsOff",  "Div2TailNum",  "Div3Airport",  "Div3AirportID",  "Div3AirportSeqID",  "Div3WheelsOn",  "Div3TotalGTime",
  "Div3LongestGTime",  "Div3WheelsOff",  "Div3TailNum",  "Div4Airport",  "Div4AirportID",  "Div4AirportSeqID",  "Div4WheelsOn",  "Div4TotalGTime",
  "Div4LongestGTime",  "Div4WheelsOff",  "Div4TailNum",  "Div5Airport",  "Div5AirportID",  "Div5AirportSeqID",  "Div5WheelsOn",  "Div5TotalGTime",
  "Div5LongestGTime",  "Div5WheelsOff",  "Div5TailNum"
FROM "ext"
PARTITIONED BY YEAR'''

display.run_task(sql)
sql_client.wait_until_ready('flights_year')

Loading data, status:[SUCCESS]: 100%|██████████| 100.0/100.0 [00:41<00:00,  2.41it/s]            


Here's the link to view the segments for this one on [the Apache Druid console in the Segments view](http://localhost:8888/unified-console.html#segments/datasource~flights_year).

Notice that now we have all the rows in a single segment and that it has over 500,000 rows. Normally this would still be a small segment, but our target for this discussion is 50,000. This solution puts us at 10x.
![](assets/segments-yearly.png)


### Almost there...
Given that there is a single month of data in this example, MONTH will not be any different than YEAR, DAY will be too granular as it will be 30x smaller and we need about 10x. We could try to use `P3D` to group segments into 3 day intervals and achieve our target. 

We can also just alter the execution parameters to force the partitioning of the large segment into the desired size by essentially cutting it into multiple segments 50,000 rows at a time, this is done with the query context parameter `rowsPerSegment`:

In [8]:
sql='''
REPLACE INTO "flights_year_50k" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://static.imply.io/example-data/flight_on_time/flights/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11.csv.zip"]}',
    '{"type":"csv","findColumnsFromHeader":true}'
  )
) EXTEND ("depaturetime" VARCHAR, "arrivalime" VARCHAR, "Year" BIGINT, "Quarter" BIGINT, "Month" BIGINT, "DayofMonth" BIGINT, "DayOfWeek" BIGINT, "FlightDate" VARCHAR, "Reporting_Airline" VARCHAR, "DOT_ID_Reporting_Airline" BIGINT, "IATA_CODE_Reporting_Airline" VARCHAR, "Tail_Number" VARCHAR, "Flight_Number_Reporting_Airline" BIGINT, "OriginAirportID" BIGINT, "OriginAirportSeqID" BIGINT, "OriginCityMarketID" BIGINT, "Origin" VARCHAR, "OriginCityName" VARCHAR, "OriginState" VARCHAR, "OriginStateFips" BIGINT, "OriginStateName" VARCHAR, "OriginWac" BIGINT, "DestAirportID" BIGINT, "DestAirportSeqID" BIGINT, "DestCityMarketID" BIGINT, "Dest" VARCHAR, "DestCityName" VARCHAR, "DestState" VARCHAR, "DestStateFips" BIGINT, "DestStateName" VARCHAR, "DestWac" BIGINT, "CRSDepTime" BIGINT, "DepTime" BIGINT, "DepDelay" BIGINT, "DepDelayMinutes" BIGINT, "DepDel15" BIGINT, "DepartureDelayGroups" BIGINT, "DepTimeBlk" VARCHAR, "TaxiOut" BIGINT, "WheelsOff" BIGINT, "WheelsOn" BIGINT, "TaxiIn" BIGINT, "CRSArrTime" BIGINT, "ArrTime" BIGINT, "ArrDelay" BIGINT, "ArrDelayMinutes" BIGINT, "ArrDel15" BIGINT, "ArrivalDelayGroups" BIGINT, "ArrTimeBlk" VARCHAR, "Cancelled" BIGINT, "CancellationCode" VARCHAR, "Diverted" BIGINT, "CRSElapsedTime" BIGINT, "ActualElapsedTime" BIGINT, "AirTime" BIGINT, "Flights" BIGINT, "Distance" BIGINT, "DistanceGroup" BIGINT, "CarrierDelay" BIGINT, "WeatherDelay" BIGINT, "NASDelay" BIGINT, "SecurityDelay" BIGINT, "LateAircraftDelay" BIGINT, "FirstDepTime" VARCHAR, "TotalAddGTime" VARCHAR, "LongestAddGTime" VARCHAR, "DivAirportLandings" VARCHAR, "DivReachedDest" VARCHAR, "DivActualElapsedTime" VARCHAR, "DivArrDelay" VARCHAR, "DivDistance" VARCHAR, "Div1Airport" VARCHAR, "Div1AirportID" VARCHAR, "Div1AirportSeqID" VARCHAR, "Div1WheelsOn" VARCHAR, "Div1TotalGTime" VARCHAR, "Div1LongestGTime" VARCHAR, "Div1WheelsOff" VARCHAR, "Div1TailNum" VARCHAR, "Div2Airport" VARCHAR, "Div2AirportID" VARCHAR, "Div2AirportSeqID" VARCHAR, "Div2WheelsOn" VARCHAR, "Div2TotalGTime" VARCHAR, "Div2LongestGTime" VARCHAR, "Div2WheelsOff" VARCHAR, "Div2TailNum" VARCHAR, "Div3Airport" VARCHAR, "Div3AirportID" VARCHAR, "Div3AirportSeqID" VARCHAR, "Div3WheelsOn" VARCHAR, "Div3TotalGTime" VARCHAR, "Div3LongestGTime" VARCHAR, "Div3WheelsOff" VARCHAR, "Div3TailNum" VARCHAR, "Div4Airport" VARCHAR, "Div4AirportID" VARCHAR, "Div4AirportSeqID" VARCHAR, "Div4WheelsOn" VARCHAR, "Div4TotalGTime" VARCHAR, "Div4LongestGTime" VARCHAR, "Div4WheelsOff" VARCHAR, "Div4TailNum" VARCHAR, "Div5Airport" VARCHAR, "Div5AirportID" VARCHAR, "Div5AirportSeqID" VARCHAR, "Div5WheelsOn" VARCHAR, "Div5TotalGTime" VARCHAR, "Div5LongestGTime" VARCHAR, "Div5WheelsOff" VARCHAR, "Div5TailNum" VARCHAR, "Unnamed: 109" VARCHAR))
SELECT
  TIME_PARSE("depaturetime") AS "__time",
  "arrivalime", "Year", "Quarter", "Month", "DayofMonth", "DayOfWeek", "FlightDate", "Reporting_Airline", "DOT_ID_Reporting_Airline",
  "IATA_CODE_Reporting_Airline", "Tail_Number", "Flight_Number_Reporting_Airline", 
  "OriginAirportID", "OriginAirportSeqID", "OriginCityMarketID", "Origin", "OriginCityName", "OriginState", "OriginStateFips", "OriginStateName", "OriginWac",  
  "DestAirportID", "DestAirportSeqID", "DestCityMarketID", "Dest", "DestCityName", "DestState", "DestStateFips", "DestStateName", "DestWac", 
  "CRSDepTime", "DepTime", "DepDelay", "DepDelayMinutes", "DepDel15", "DepartureDelayGroups", "DepTimeBlk",
  "TaxiOut", "WheelsOff", "WheelsOn", "TaxiIn", "CRSArrTime", "ArrTime", "ArrDelay", "ArrDelayMinutes", "ArrDel15", "ArrivalDelayGroups", "ArrTimeBlk",
  "Cancelled", "CancellationCode", "Diverted", "CRSElapsedTime", "ActualElapsedTime", "AirTime", "Flights", "Distance", "DistanceGroup", "CarrierDelay",
  "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay", "FirstDepTime", "TotalAddGTime", "LongestAddGTime", "DivAirportLandings", "DivReachedDest",
  "DivActualElapsedTime",  "DivArrDelay",  "DivDistance",  "Div1Airport",  "Div1AirportID",  "Div1AirportSeqID",  "Div1WheelsOn",  "Div1TotalGTime",
  "Div1LongestGTime",  "Div1WheelsOff",  "Div1TailNum",  "Div2Airport",  "Div2AirportID",  "Div2AirportSeqID",  "Div2WheelsOn",  "Div2TotalGTime",
  "Div2LongestGTime",  "Div2WheelsOff",  "Div2TailNum",  "Div3Airport",  "Div3AirportID",  "Div3AirportSeqID",  "Div3WheelsOn",  "Div3TotalGTime",
  "Div3LongestGTime",  "Div3WheelsOff",  "Div3TailNum",  "Div4Airport",  "Div4AirportID",  "Div4AirportSeqID",  "Div4WheelsOn",  "Div4TotalGTime",
  "Div4LongestGTime",  "Div4WheelsOff",  "Div4TailNum",  "Div5Airport",  "Div5AirportID",  "Div5AirportSeqID",  "Div5WheelsOn",  "Div5TotalGTime",
  "Div5LongestGTime",  "Div5WheelsOff",  "Div5TailNum"
FROM "ext"
PARTITIONED BY YEAR'''
# use a request so that we can specify the query context
req = sql_client.sql_request(sql)
req.add_context("rowsPerSegment", "50000")  
display.run_task(req)
sql_client.wait_until_ready('flights_year_50k')

Loading data, status:[SUCCESS]: 100%|██████████| 100.0/100.0 [00:42<00:00,  2.35it/s]            


The [segment view for this table](http://localhost:8888/unified-console.html#segments/datasource~flights_year_50k) shows 12 segments that are very close to our ideal segment size. 

But let's say that the application we are building is for year long analytics for individual airlines and some airline to airline comparisons. So let's move on to clustering...

## Clustering 

As mentioned in the introduction, time intervals of a given segment granularity can be subdivided into many segments. In the example above, we achieved this but there is no logic to the partitioning, they were just cut into 12 equally sized segments that were as close to the 50k target as possible.  

Queries on this data will be frequently filtered on `IATA_CODE_Reporting_Airline` in order to provide individual airline analytics. So instead of just splitting the segments by quantity of rows, we can reorganize the data such that we can improve pruning when filtering on a given airline or a few airlines.
We just need to add `CLUSTERED BY IATA_CODE_Reporting_Airline` to the ingestion request:

In [9]:
sql='''
REPLACE INTO "flights_year_IATA" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://static.imply.io/example-data/flight_on_time/flights/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11.csv.zip"]}',
    '{"type":"csv","findColumnsFromHeader":true}'
  )
) EXTEND ("depaturetime" VARCHAR, "arrivalime" VARCHAR, "Year" BIGINT, "Quarter" BIGINT, "Month" BIGINT, "DayofMonth" BIGINT, "DayOfWeek" BIGINT, "FlightDate" VARCHAR, "Reporting_Airline" VARCHAR, "DOT_ID_Reporting_Airline" BIGINT, "IATA_CODE_Reporting_Airline" VARCHAR, "Tail_Number" VARCHAR, "Flight_Number_Reporting_Airline" BIGINT, "OriginAirportID" BIGINT, "OriginAirportSeqID" BIGINT, "OriginCityMarketID" BIGINT, "Origin" VARCHAR, "OriginCityName" VARCHAR, "OriginState" VARCHAR, "OriginStateFips" BIGINT, "OriginStateName" VARCHAR, "OriginWac" BIGINT, "DestAirportID" BIGINT, "DestAirportSeqID" BIGINT, "DestCityMarketID" BIGINT, "Dest" VARCHAR, "DestCityName" VARCHAR, "DestState" VARCHAR, "DestStateFips" BIGINT, "DestStateName" VARCHAR, "DestWac" BIGINT, "CRSDepTime" BIGINT, "DepTime" BIGINT, "DepDelay" BIGINT, "DepDelayMinutes" BIGINT, "DepDel15" BIGINT, "DepartureDelayGroups" BIGINT, "DepTimeBlk" VARCHAR, "TaxiOut" BIGINT, "WheelsOff" BIGINT, "WheelsOn" BIGINT, "TaxiIn" BIGINT, "CRSArrTime" BIGINT, "ArrTime" BIGINT, "ArrDelay" BIGINT, "ArrDelayMinutes" BIGINT, "ArrDel15" BIGINT, "ArrivalDelayGroups" BIGINT, "ArrTimeBlk" VARCHAR, "Cancelled" BIGINT, "CancellationCode" VARCHAR, "Diverted" BIGINT, "CRSElapsedTime" BIGINT, "ActualElapsedTime" BIGINT, "AirTime" BIGINT, "Flights" BIGINT, "Distance" BIGINT, "DistanceGroup" BIGINT, "CarrierDelay" BIGINT, "WeatherDelay" BIGINT, "NASDelay" BIGINT, "SecurityDelay" BIGINT, "LateAircraftDelay" BIGINT, "FirstDepTime" VARCHAR, "TotalAddGTime" VARCHAR, "LongestAddGTime" VARCHAR, "DivAirportLandings" VARCHAR, "DivReachedDest" VARCHAR, "DivActualElapsedTime" VARCHAR, "DivArrDelay" VARCHAR, "DivDistance" VARCHAR, "Div1Airport" VARCHAR, "Div1AirportID" VARCHAR, "Div1AirportSeqID" VARCHAR, "Div1WheelsOn" VARCHAR, "Div1TotalGTime" VARCHAR, "Div1LongestGTime" VARCHAR, "Div1WheelsOff" VARCHAR, "Div1TailNum" VARCHAR, "Div2Airport" VARCHAR, "Div2AirportID" VARCHAR, "Div2AirportSeqID" VARCHAR, "Div2WheelsOn" VARCHAR, "Div2TotalGTime" VARCHAR, "Div2LongestGTime" VARCHAR, "Div2WheelsOff" VARCHAR, "Div2TailNum" VARCHAR, "Div3Airport" VARCHAR, "Div3AirportID" VARCHAR, "Div3AirportSeqID" VARCHAR, "Div3WheelsOn" VARCHAR, "Div3TotalGTime" VARCHAR, "Div3LongestGTime" VARCHAR, "Div3WheelsOff" VARCHAR, "Div3TailNum" VARCHAR, "Div4Airport" VARCHAR, "Div4AirportID" VARCHAR, "Div4AirportSeqID" VARCHAR, "Div4WheelsOn" VARCHAR, "Div4TotalGTime" VARCHAR, "Div4LongestGTime" VARCHAR, "Div4WheelsOff" VARCHAR, "Div4TailNum" VARCHAR, "Div5Airport" VARCHAR, "Div5AirportID" VARCHAR, "Div5AirportSeqID" VARCHAR, "Div5WheelsOn" VARCHAR, "Div5TotalGTime" VARCHAR, "Div5LongestGTime" VARCHAR, "Div5WheelsOff" VARCHAR, "Div5TailNum" VARCHAR, "Unnamed: 109" VARCHAR))
SELECT
  TIME_PARSE("depaturetime") AS "__time",
  "arrivalime", "Year", "Quarter", "Month", "DayofMonth", "DayOfWeek", "FlightDate", "Reporting_Airline", "DOT_ID_Reporting_Airline",
  "IATA_CODE_Reporting_Airline", "Tail_Number", "Flight_Number_Reporting_Airline", 
  "OriginAirportID", "OriginAirportSeqID", "OriginCityMarketID", "Origin", "OriginCityName", "OriginState", "OriginStateFips", "OriginStateName", "OriginWac",  
  "DestAirportID", "DestAirportSeqID", "DestCityMarketID", "Dest", "DestCityName", "DestState", "DestStateFips", "DestStateName", "DestWac", 
  "CRSDepTime", "DepTime", "DepDelay", "DepDelayMinutes", "DepDel15", "DepartureDelayGroups", "DepTimeBlk",
  "TaxiOut", "WheelsOff", "WheelsOn", "TaxiIn", "CRSArrTime", "ArrTime", "ArrDelay", "ArrDelayMinutes", "ArrDel15", "ArrivalDelayGroups", "ArrTimeBlk",
  "Cancelled", "CancellationCode", "Diverted", "CRSElapsedTime", "ActualElapsedTime", "AirTime", "Flights", "Distance", "DistanceGroup", "CarrierDelay",
  "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay", "FirstDepTime", "TotalAddGTime", "LongestAddGTime", "DivAirportLandings", "DivReachedDest",
  "DivActualElapsedTime",  "DivArrDelay",  "DivDistance",  "Div1Airport",  "Div1AirportID",  "Div1AirportSeqID",  "Div1WheelsOn",  "Div1TotalGTime",
  "Div1LongestGTime",  "Div1WheelsOff",  "Div1TailNum",  "Div2Airport",  "Div2AirportID",  "Div2AirportSeqID",  "Div2WheelsOn",  "Div2TotalGTime",
  "Div2LongestGTime",  "Div2WheelsOff",  "Div2TailNum",  "Div3Airport",  "Div3AirportID",  "Div3AirportSeqID",  "Div3WheelsOn",  "Div3TotalGTime",
  "Div3LongestGTime",  "Div3WheelsOff",  "Div3TailNum",  "Div4Airport",  "Div4AirportID",  "Div4AirportSeqID",  "Div4WheelsOn",  "Div4TotalGTime",
  "Div4LongestGTime",  "Div4WheelsOff",  "Div4TailNum",  "Div5Airport",  "Div5AirportID",  "Div5AirportSeqID",  "Div5WheelsOn",  "Div5TotalGTime",
  "Div5LongestGTime",  "Div5WheelsOff",  "Div5TailNum"
FROM "ext"
PARTITIONED BY YEAR
CLUSTERED BY IATA_CODE_Reporting_Airline
'''
# use a request so that we can specify the query context
req = sql_client.sql_request(sql)
req.add_context("rowsPerSegment", "50000")  # we still want to target out ideal of 50k per segment
display.run_task(req)
sql_client.wait_until_ready('flights_year_IATA')

Loading data, status:[SUCCESS]: 100%|██████████| 100.0/100.0 [00:43<00:00,  2.29it/s]            


The [segment view for this table](http://localhost:8888/unified-console.html#segments/datasource~flights_year_IATA) shows 12 segments that are still very close to our ideal segment size, but now they have some new metadata based on the clustering columns in the `Shard Spec` column. It shows which range of values of the IATA_CODE_Reporting_Airline column are available in the segment. This is very useful for pruning when filtering on that column because Druid can prune to 1 or 2 segment files when looking for a single airline.

![](assets/segments-iata.png)

## Query Performance
Now that the data has been loaded using a few different partitioning strategies, you can use a test query to determine which one is the best.
In order to control the experiment we will turn cacheing off for the results of the query and measure the same query with each table.

### The Query
It is well known that flight traffic follows a weekly pattern as well as a seasonal pattern. Given that we only have a month, let's take a look at the maximum and average delay for a single airline by day of the week so that we can see the pattern and how it affects delays. We'll only consider the departure delay for now and only use delays greater than 2 minutes as "delayed".

One caveat about measuring performance on a laptop. The results obtained here will not necessarily be the best when the data grows and is distributed on a multi-node cluster. The intent here is to provide a way of thinking about optimizing the data. In a real cluster, it is likely that the results will vary and a different partitioning and clustering strategy will provide better results.

The following cell defines a function we can use to measure performance:

In [36]:
from datetime import datetime
from statistics import mean 

def measure_query( sql: str, iterations: int ):
    req = sql_client.sql_request(sql)
    req.add_context("populateCache", "false")  # run without cacheing results to get a real sense of performance
    req.add_context("useCache", "false")  # do not use cached results
    stats = []
    while (iterations>0):
      start = datetime.now()
      sql_client.sql(req)
      end = datetime.now()
      stats.append( (end - start).total_seconds() * 10**3 ) # add run time in milliseconds
      iterations -=1
    return f"Results = avg:{mean(stats)} ms   min:{min(stats)} ms  max:{max(stats)} ms"

Run the test SQL to see that in the month of November in 2005, American Airlines had the highest percentage of delayed flights on Sundays(7) although the worst average delay occurs on Fridays(5):

In [37]:
sql = '''
SELECT EXTRACT( DOW FROM __time) as day_of_week, 
       AVG(DepDelay) FILTER ( WHERE DepDelay>120) as avg_delay,
       MAX(DepDelay) FILTER ( WHERE DepDelay>120) as max_delay,
       ROUND(COUNT(1) FILTER ( WHERE DepDelay>120) * 100.0 / COUNT(1), 1) as percent_delayed
FROM {}
WHERE "IATA_CODE_Reporting_Airline" = 'AA'
GROUP BY 1 
ORDER BY 1 ASC
'''

display.sql(sql.format("flights_hour"))

day_of_week,avg_delay,max_delay,percent_delayed
1,186,1038,0.8
2,170,452,1.3
3,175,921,0.8
4,173,339,0.5
5,239,1210,0.7
6,227,764,0.5
7,186,640,1.8


### Performance Results

In [39]:
# run 100 queries with each table
use_table="flights_hour"
print (f"{use_table} {measure_query(sql.format(use_table), 100)}")

use_table="flights_year"
print (f"{use_table} {measure_query(sql.format(use_table), 100)}")

use_table="flights_year_50k"
print (f"{use_table} {measure_query(sql.format(use_table), 100)}")

use_table="flights_year_IATA"
print (f"{use_table} {measure_query(sql.format(use_table), 100)}")

flights_hour Results = avg:26.85229 ms   min:24.956 ms  max:38.013999999999996 ms
flights_year Results = avg:15.08775 ms   min:13.826 ms  max:18.481 ms
flights_year_50k Results = avg:12.59755 ms   min:11.321 ms  max:17.597 ms
flights_year_IATA Results = avg:12.5657 ms   min:11.093 ms  max:15.056999999999999 ms


### Single Concurrency Results

Given that the characteristics of your laptop or other execution environment are likely different than the `Apple M2 Pro` that this was built on, your results will vary, my results were:
```
flights_hour Results      = avg:28.848 ms   min:25.198 ms   max:64.379 ms
flights_year Results      = avg:15.451 ms   min:13.891 ms   max:17.940 ms
flights_year_50k Results  = avg:12.844 ms   min:11.594 ms   max:15.130 ms
flights_year_IATA Results = avg:12.574 ms   min:11.221 ms   max:15.204 ms
```

- `flights_hour` - With segment granularity of hour, there are approximately 30*24= 720 segment files to scan, there is no pruning, so even though the segments are tiny, each one is examined separately and the results are then merged, so doing that 720 times makes this the slowest option.
- `flights_year` - Here we are using a single segment, it is much larger at 500k+ rows, but Druid can still use the indexes associated to all dimension columns to process this pretty fast and it only does it once, so this is faster then hourly. This shows how the number of segments needed for a query is a factor for performance. Less segments to process tends to improve perfomance, but not always.
- `flights_year_50k` - Now each query had to process twelve 50k segments, but given that the historical does this concurrently on multiple threads, the parallelism is enough to improve over the single 500k segment of the `flights_year` table. 
- `flights_year_IATA` - Given the shard spec of the 12 resulting segments, we can see that there are rows with IATA Code "AA" in 2 of the segments. They are both processed in parallel and there are less segments to process, so this results in the fastest. 

### High Concurrency Results

Above, the results for `flights_year_IATA` seem very similar to the `flights_year_50k` results, but a real system processes multiple queries at once. Expanding the test to run with higher concurrency will tell us more. The expectation is that the organization of the data that requires less work per query will ultimately be better for higher concurrency.

We'll need another set of functions to drive the queries in parallel.


In [63]:
from threading import Thread
from datetime import datetime
from statistics import mean, median


# custom thread
class QueryThread(Thread):
    # constructor
    def __init__(self, sql: str, iterations: int ):
        # execute the base constructor
        Thread.__init__(self)
        # set a default value
        self.stats = []
        self.sql = sql
        self.iterations = iterations
 
    # function executed in a new thread
    def run(self):
        self.stats = measure_query( self.sql, self.iterations)
        
    def measure_query( sql: str, iterations: int ):
        req = sql_client.sql_request(sql)
        req.add_context("populateCache", "false")  # run without cacheing results to get a real sense of performance
        req.add_context("useCache", "false")  # do not use cached results
        stats=[]
        while (iterations>0):
          start = datetime.now()
          sql_client.sql(req)
          end = datetime.now()
          stats.append( (end - start).total_seconds() * 10**3 ) # add run time in milliseconds
          iterations -=1
        return stats

def test_in_parallel( sql: str, numThreads: int, iterations: int):
    threads=[]
    while numThreads>0:
        try:
            thrd = QueryThread( sql, iterations)
            thrd.start()
            threads.append(thrd)
        except:
           print ("Error: unable to start thread")
        numThreads-=1
    total_stats = []
    for thrd in threads:
        thrd.join()
        total_stats.extend(thrd.stats)
    return f"Results = count:{len(total_stats)}  avg:{median(total_stats)} ms   min:{min(total_stats)} ms  max:{max(total_stats)} ms"  
    

In [66]:
use_table="flights_year_50k"
print (f"{use_table} {test_in_parallel(sql.format(use_table), 2, 50)}")

use_table="flights_year_IATA"
print (f"{use_table} {test_in_parallel(sql.format(use_table), 2, 50)}")

# ADD VARIABLE FILTERS to get more compelling results



flights_year_50k Results = count:100  avg:12.8925 ms   min:11.422 ms  max:30.237 ms
flights_year_IATA Results = count:100  avg:12.638 ms   min:11.555 ms  max:15.449 ms


## Clean up

Run the following cell to remove the XXX used in this notebook from the database.

In [7]:
druid.datasources.drop("flights_hour")
druid.datasources.drop("flights_year")
druid.datasources.drop("flights_year_50k")
druid.datasources.drop("flights_year_IATA")

## Summary

* You learned this
* Remember this

## Learn more

* Try this out on your own data
* Solve for problem X that is't covered here
* Read docs pages
* Watch or read something cool from the community
* Do some exploratory stuff on your own